In [2]:
import os
import sys
import pandas as pd

# 添加项目根目录到 Python 路径
project_root = '/date0/crwu/zjts_foodname'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from packages.file_deal import read_dataset

2025-10-14 10:30:07 - file_deal - INFO - [my_logger.py:103] - 日志系统初始化完成，日志文件: /date0/crwu/zjts_foodname/logs/20251014_103007_file_deal.log
2025-10-14 10:30:07 - file_deal - INFO - [my_logger.py:104] - 脚本名称: file_deal


# 读各个数据源，包括分类数据库， 食物分类层级， 食物标签

In [3]:
from collections import defaultdict

csv_root = r'/date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C20/'
csv_data = r'/date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C10/merged_results.csv'
csv_data = r'/date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C20/food_name_database_V100R25C20.csv'
food_depth_path = r'/date0/crwu/zjts_foodname/data/classification/data/food_depth.csv'
food_tags_path = r'/date0/crwu/zjts_foodname/data/classification/data/food_tags.csv'

depth_df = read_dataset(food_depth_path)
tags_df = read_dataset(food_tags_path)
df = read_dataset(csv_data)
rule_dict = defaultdict(set)
for row in depth_df.itertuples():
    _, *levels = tuple(row)
    rule_dict['第一级别'].add(','.join(levels))
    rule_dict['第二级别'].add(','.join(levels[1:]))
    rule_dict['第三级别'].add(','.join(levels[2:]))
    rule_dict['第四级别'].add(','.join(levels[3:]))

    rule_dict['第一级别'].add(levels[0])
    rule_dict['第二级别'].add(levels[1])
    rule_dict['第三级别'].add(levels[2])
    rule_dict['第四级别'].add(levels[3])

for column in tags_df.columns:
    for val in tags_df[column]:
        if isinstance(val, float) and not pd.isna(val):
            val = str(int(val))
        if not pd.isna(val):
            rule_dict[column].add(val)
print(rule_dict) 



2025-10-14 10:30:09 - file_deal - INFO - [file_deal.py:151] - Loading dataset from CSV: /date0/crwu/zjts_foodname/data/classification/data/food_depth.csv ...
2025-10-14 10:30:09 - file_deal - INFO - [file_deal.py:151] - Loading dataset from CSV: /date0/crwu/zjts_foodname/data/classification/data/food_tags.csv ...
2025-10-14 10:30:09 - file_deal - INFO - [file_deal.py:151] - Loading dataset from CSV: /date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C20/food_name_database_V100R25C20.csv ...
defaultdict(<class 'set'>, {'第一级别': {'A. 富含优质蛋白质的菜品,A1. 红肉类,猪肉,肉丸/肉饼', 'B. 以碳水化合物食材为主的菜品,B1. 精制谷物类,精米饭/米粉/米制品,粥', 'A. 富含优质蛋白质的菜品,A2. 禽肉类,鸡肉,鸡腿', 'A. 富含优质蛋白质的菜品,A1. 红肉类,猪肉,猪颈肉', 'A. 富含优质蛋白质的菜品,A5. 内脏类,禽类内脏,肝（鸡肝、鸭肝等）', 'A. 富含优质蛋白质的菜品,A1. 红肉类,猪肉,肉丝', 'A. 富含优质蛋白质的菜品,A1. 红肉类,牛肉,牛肉丸', 'A. 富含优质蛋白质的菜品,A3. 鱼类,高脂肪鱼类,高脂肪鱼类', 'A. 富含优质蛋白质的菜品,A6. 蛋制品类,未处理的蛋,鹅蛋', 'A. 富含优质蛋白质的菜品,A6. 蛋制品类,经过预处理的蛋,皮蛋/松花蛋', 'H. 饮品,H. 饮品,酒精饮品,中式白酒', 'A. 富含优质蛋白质的菜品,A1. 红肉类,羊肉,羊蝎子', 'A. 富含优质蛋白质的菜品,A3. 鱼类,低

# 合并表格用的

In [ ]:
# csv_data_2 = r'/date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C20/merged_results.csv'

# depth_df = read_dataset(food_depth_path)
# tags_df = read_dataset(food_tags_path)
# df_2 = read_dataset(csv_data_2)
# df['营养建议分类'] = df_2['营养建议分类']
# df['早中晚餐消费场景'] = df_2['早中晚餐消费场景']
# df['食用频率'] = df_2['食用频率']
# df['熟悉程度'] = df_2['熟悉程度']
# print(df.iloc[0:10,0:15])
# df.to_csv(os.path.join(csv_root, 'food_name_database_V100R25C20.csv'), index=False)

In [4]:
failed_vals_dict = {}
for column in df.columns:
    if column == '菜品名':
        continue    
    if column == '主要食材种类' or column == '营养建议分类':
        val_list = df[column]
        valid_values = set()
        for vals in val_list:
            for val in vals.split('/'):
                valid_values.add(val)
    else:
        valid_values = set(df[column])
    for val in list(valid_values):
        if (val.startswith('(') and val.endswith(')')) or val == '不适用':
            valid_values.remove(val)
    print(f'{column}共有{len(valid_values)}个取值')
    not_qulified_vals = valid_values - rule_dict[column]
    print(f'不在rule中的值有{len(not_qulified_vals)}个: {not_qulified_vals}')
    if not_qulified_vals:    
        failed_vals_dict[column] = list(not_qulified_vals)
print(failed_vals_dict)

第一级别共有7个取值
不在rule中的值有0个: set()
第二级别共有17个取值
不在rule中的值有0个: set()
第三级别共有52个取值
不在rule中的值有0个: set()
第四级别共有155个取值
不在rule中的值有0个: set()
主要食材种类共有51个取值
不在rule中的值有0个: set()
烹饪方式共有22个取值
不在rule中的值有0个: set()
菜系（地域）分类共有21个取值
不在rule中的值有0个: set()
消费场景共有6个取值
不在rule中的值有0个: set()
营养建议分类共有5个取值
不在rule中的值有0个: set()
记忆/推理菜共有2个取值
不在rule中的值有0个: set()
早中晚餐消费场景共有5个取值
不在rule中的值有0个: set()
食用频率共有5个取值
不在rule中的值有0个: set()
熟悉程度共有5个取值
不在rule中的值有0个: set()
{}


# 淡水鱼和海水鱼是手动删掉合并的，切记切记

In [5]:

failed_row_index_set = set()
for column, failed_val_list in failed_vals_dict.items():
    if column != '主要食材种类':
        failed_df = df[df[column].isin(failed_val_list)]
        print(f'打印"{column}"的失败项,共计{len(failed_df)}个')
        print(failed_df)
        print('*' * 50)
        for index in failed_df.index:
            failed_row_index_set.add(index)
    else:
        count = 0
        for row in df.iterrows():
            # print()
            # break
            for val in failed_val_list:
                if val in row[1].iloc[5].split('/'):
                    count += 1
                    print(row[1])
                    failed_row_index_set.add(row[0])
                    break
        print(f'打印"{column}"的失败项,共计{count}个')
        # print(failed_df)
        print('*' * 50)


In [6]:
fail_count = 0
for row in df.iterrows():
    lv1, lv2, lv3, lv4 = tuple(row[1].iloc[1:5])
    if ",".join(list(row[1].iloc[1:5])) not in rule_dict["第一级别"] and \
    lv1 != '(非菜品)' and \
    lv1 != '不适用' and \
    lv2 != '不适用' and \
    lv3 != '不适用' and \
    lv4 != '不适用' and \
        True:
        fail_count += 1
        print(list(row[1].iloc[0:5]))
        failed_row_index_set.add(row[0])
print(f'{fail_count}条数据层级不对')

0条数据层级不对


In [7]:
print(failed_row_index_set)
print(f'失败行共{len(failed_row_index_set)}个')
failed_files_set = set()
for failed_row_index in list(failed_row_index_set):
    failed_files_set.add(failed_row_index//5)
print(failed_files_set)
print(f'失败文件共{len(failed_files_set)}个')

    

set()
失败行共0个
set()
失败文件共0个


# 删除有问题的.pk文件然后重跑打标签代码(01_classify_food.py)，对有问题的部分重新打标

In [221]:
import os
pk_file_root = r'/date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C10'
for failed_pk_index in list(failed_files_set):
    try:        
        # os.remove(os.path.join(pk_file_root, f'{failed_pk_index}.pk'))
        print(f'删除失败文件{failed_pk_index}.pk')
    except:
        pass